# Лабораторная работа 5

   по теме
                 
   **Распознавание объектов на фотографиях**
   
   ****

   Выполнил студент

   Группы БСТ1801

   Харатишвили Заза

In [18]:
#Импорты
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten

from tensorflow.keras.utils import to_categorical
import numpy as np

from keras.utils import np_utils

In [19]:
batch_size = 32 # на каждой итерации одновременно обрабатываем 32 обучающих образца
num_epochs = 5 # количество итераций
kernel_size = 3 # размер ядра в свертывающих слоях
pool_size = 2 # размер подвыборки в слоях подвыборки
conv_depth_1 = 32 # количество ядер в свертывающих слоях
conv_depth_2 = 64 # увеличиваем в два раза после слоя подвыборки 
drop_prob_1 = 0.25 # убираем нейроны после слоя подвыборки с вероятностью 0.25
drop_prob_2 = 0.5 # убираем нейроны после полносвязного слоя с вероятностью 0.5
hidden_size = 512 # количество нейронов в полносвязном слое

In [24]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # получаем данные CIFAR-10 

# 50000 данных для обучения, 10000 для тестирования, 10 классов картинок
num_train, depth, height, width = X_train.shape
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0]

# Приведем к виду с плавающей точкой
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
# Преобразуем интетнсивность пикселей до [0 ,1]
X_train /= np.max(X_train)
X_test /= np.max(X_train) #делим на обучающее, тк нельзя показывать алгоритмам тестовые данные до окончания 
# процесса обучения

Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

In [13]:
inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования
model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/5
1407/1407 [==============================] - 51s 36ms/step - loss: 1.5254 - accuracy: 0.4466 - val_loss: 1.1101 - val_accuracy: 0.6120
Epoch 2/5
1407/1407 [==============================] - 52s 37ms/step - loss: 1.1210 - accuracy: 0.6003 - val_loss: 0.9594 - val_accuracy: 0.6646
Epoch 3/5
1407/1407 [==============================] - 51s 37ms/step - loss: 0.9691 - accuracy: 0.6581 - val_loss: 0.8483 - val_accuracy: 0.7046
Epoch 4/5
1407/1407 [==============================] - 52s 37ms/step - loss: 0.8792 - accuracy: 0.6908 - val_loss: 0.7852 - val_accuracy: 0.7324
Epoch 5/5
313/313 [==============================] - 2s 8ms/step - loss: 132.9777 - accuracy: 0.5315


[132.9777069091797, 0.531499981880188]

Исследуем сеть без слоя Dropout

In [27]:
inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)

# Conv [64] -> Conv [64] -> Pool
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(pool_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)

# Now flatten to 1D, apply FC -> ReLU -> softmax
flat = Flatten()(conv_4) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования
model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/5
1407/1407 [==============================] - 73s 52ms/step - loss: 1.4598 - accuracy: 0.4678 - val_loss: 1.0523 - val_accuracy: 0.6272
Epoch 2/5
1407/1407 [==============================] - 72s 51ms/step - loss: 1.0188 - accuracy: 0.6425 - val_loss: 0.8652 - val_accuracy: 0.6956
Epoch 3/5
1407/1407 [==============================] - 71s 50ms/step - loss: 0.8220 - accuracy: 0.7119 - val_loss: 0.7840 - val_accuracy: 0.7302
Epoch 4/5
1407/1407 [==============================] - 71s 51ms/step - loss: 0.6785 - accuracy: 0.7636 - val_loss: 0.7252 - val_accuracy: 0.7540
Epoch 5/5
313/313 [==============================] - 3s 10ms/step - loss: 180.0096 - accuracy: 0.5340


[180.0095977783203, 0.5339999794960022]

Уменьшим размер ядра свертки в два раза

In [29]:
conv_depth_1 = 16 # количество ядер в свертывающих слоях

inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования
model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/5
1407/1407 [==============================] - 40s 28ms/step - loss: 1.5656 - accuracy: 0.4236 - val_loss: 1.1850 - val_accuracy: 0.5822
Epoch 2/5
1407/1407 [==============================] - 40s 28ms/step - loss: 1.1669 - accuracy: 0.5827 - val_loss: 1.0015 - val_accuracy: 0.6494
Epoch 3/5
1407/1407 [==============================] - 39s 28ms/step - loss: 1.0074 - accuracy: 0.6438 - val_loss: 0.8932 - val_accuracy: 0.6952
Epoch 4/5
1407/1407 [==============================] - 39s 28ms/step - loss: 0.9041 - accuracy: 0.6825 - val_loss: 0.7929 - val_accuracy: 0.7228
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 113.5296 - accuracy: 0.5271


[113.52964782714844, 0.5271000266075134]

Увеличим размер ядра свертки в два раза

In [30]:
conv_depth_1 = 64 # количество ядер в свертывающих слоях

inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования
model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/5
1407/1407 [==============================] - 83s 59ms/step - loss: 1.5374 - accuracy: 0.4342 - val_loss: 1.1313 - val_accuracy: 0.6002
Epoch 2/5
1407/1407 [==============================] - 84s 60ms/step - loss: 1.1463 - accuracy: 0.5920 - val_loss: 0.9456 - val_accuracy: 0.6706
Epoch 3/5
1407/1407 [==============================] - 85s 60ms/step - loss: 0.9829 - accuracy: 0.6517 - val_loss: 0.8523 - val_accuracy: 0.7018
Epoch 4/5
1407/1407 [==============================] - 85s 61ms/step - loss: 0.8866 - accuracy: 0.6872 - val_loss: 0.7783 - val_accuracy: 0.7288
Epoch 5/5
313/313 [==============================] - 4s 11ms/step - loss: 114.0328 - accuracy: 0.5453


[114.03275299072266, 0.5453000068664551]